In [ ]:
# Add project src to path.
import set_path

import numpy as np

from src.utils.labels import Labels
from src.scrapers.ams_bgt_scraper import scrape_amsterdam_bgt, parse_polygons
from src.utils.las_utils import read_las, label_and_save_las, get_bbox_from_las_file
from src.utils.clip_utils import poly_clip
from src.region_growing.label_connected_comp import LabelConnectedComp

In [ ]:
in_file = '../datasets/pointcloud/grown_2397_9705.las'
out_file = '../datasets/pointcloud/cars_2397_9705.las'

In [ ]:
pointcloud = read_las(in_file)
las_labels = pointcloud.label
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T

In [ ]:
exclude_labels = (Labels.GROUND, Labels.BUILDING)

In [ ]:
bbox = get_bbox_from_las_file(in_file)

In [ ]:
# TODO parse_polygons moeten we als CSV opslaan
joined_road_polygons = []
for layer in ["BGT_WGL_rijbaan_lokale_weg", "BGT_WGL_parkeervlak"]:
    json_content = scrape_amsterdam_bgt(layer, bbox=bbox)
    road_polygons = parse_polygons(json_content)
    joined_road_polygons += road_polygons

In [ ]:
m_above_ground = 3

# TODO dit gaat alleen werken voor een tegel met weinig heuvels
avg_ground_height = np.average(pointcloud.z[las_labels == Labels.GROUND])
max_z_thresh = avg_ground_height + m_above_ground
print(max_z_thresh)

# TODO Daarom miss beter het gemiddelde van een stukje road berekenen.
if joined_road_polygons:
    road_mask = poly_clip(points, joined_road_polygons[0]) # Take one polygon
    road_ground_mask = np.logical_and(las_labels == Labels.GROUND,road_mask)
    avg_ground_height = np.average(pointcloud.z[road_ground_mask])
    max_z_thresh = avg_ground_height + m_above_ground
    print(max_z_thresh)

In [ ]:
road_mask

In [ ]:
# TODO dit verwerken in de pipeline?
lcc = LabelConnectedComp(Labels.CAR, exclude_labels, octree_level=10,
                         min_component_size=100)
lcc._set_mask(las_labels)
lcc._convert_input_cloud(points)
lcc._label_connected_comp()
las_labels = lcc._overlapping_polygons(joined_road_polygons, max_z_thresh, 
                                       min_area_thresh=6, max_area_thresh=16)

In [ ]:
label_and_save_las(pointcloud, las_labels, out_file)